# Python web scraper script

1. installing requirements

In [111]:
import sys
# you can also directly run this command in CLI 
# installing pandas
!{sys.executable} -m pip install pandas
# installing bs4
!{sys.executable} -m pip install bs4
# for exporting in excel sheet
!{sys.executable} -m pip install openpyxl
!{sys.executable} -m pip install lxml





[notice] A new release of pip available: 22.3 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


2. Importing packages and libraries

In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import html5lib
import lxml
import requests, time, datetime, re, random

In [4]:
url = "https://en.m.wikipedia.org/wiki/List_of_best-selling_Nintendo_Switch_video_games"
headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"}
response = requests.get(url, headers=headers)
# check if the status is ok 
print(response.status_code)
# print(response.text)

200


In [5]:
soup = BeautifulSoup(response.text, 'html.parser')
table_html = soup.find('table' , {'class':"wikitable"})
table_html

<table class="wikitable plainrowheaders sortable"><caption>List of best-selling Nintendo Switch video games
</caption>
<tbody><tr><th scope="col">Rank
</th>
<th scope="col">Title
</th>
<th scope="col">Copies sold
</th>
<th scope="col">As of
</th>
<th scope="col">Release date<sup class="reference" id="cite_ref-5"><a href="#cite_note-5">[a]</a></sup></th>
<th scope="col">Genre(s)
</th>
<th scope="col">Developer(s)
</th>
<th scope="col">Publisher(s)
</th></tr><tr><td style="text-align: center;">1
</td>
<th scope="row"><i><a class="mw-redirect" href="/wiki/Mario_Kart_8_Deluxe" title="Mario Kart 8 Deluxe">Mario Kart 8 Deluxe</a></i>
</th>
<td><span data-sort-value="7001537900000000000♠">53.79</span><span class="nowrap"> </span>million<sup class="reference" id="cite_ref-titlesales_4-1"><a href="#cite_note-titlesales-4">[4]</a></sup></td>
<td><span data-sort-value="000000002023-03-31-0000" style="white-space:nowrap">March 31, 2023</span>
</td>
<td><span data-sort-value="000000002017-04-28-000

In [7]:
table_dataframe = pd.read_html(str(table_html))[0]
table_dataframe

,Rank,Title,Copies sold,As of,Release date[a],Genre(s),Developer(s),Publisher(s)
0,1,Mario Kart 8 Deluxe,53.79 million[4],"March 31, 2023","April 28, 2017",Kart racing,Nintendo EPD,Nintendo
1,2,Animal Crossing: New Horizons,42.21 million[4],"March 31, 2023","March 20, 2020",Social simulation,Nintendo EPD,Nintendo
2,3,Super Smash Bros. Ultimate,31.09 million[4],"March 31, 2023","December 7, 2018",Fighting,Bandai Namco StudiosSora Ltd.,Nintendo
3,4,The Legend of Zelda: Breath of the Wild,29.81 million[4],"March 31, 2023","March 3, 2017",Action-adventure,Nintendo EPD,Nintendo
4,5,Pokémon Sword and Shield,25.82 million[4],"March 31, 2023","November 15, 2019",Role-playing,Game Freak,The Pokémon CompanyNintendo
...,...,...,...,...,...,...,...,...
73,72,Fitness Boxing,1 million[25],"September 8, 2020","December 20, 2018",Exergamerhythm,Imagineer,JP: ImagineerNA/PAL: Nintendo
74,72,Fitness Boxing 2: Rhythm and Exercise,1 million[26],"December 9, 2021","December 4, 2020",Exergamerhythm,Imagineer,JP: ImagineerNA/PAL: Nintendo
75,72,Shin Megami Tensei V,1 million[27],"April 18, 2022","November 11, 2021",Role-playing,Atlus,JP: AtlusNA: SegaPAL: Nintendo
76,72,Story of Seasons: Pioneers of Olive Town,1 million[28],"November 18, 2021","February 25, 2021",Simulationrole-playing,Marvelous,Xseed Games


### Formatting the price, dates 

In [74]:
# function to convert the string copies sold value to float
def convert_to_float(copies_sold):
    #  remove the string value using regex
    clean_value = re.sub(r'million(\[[a-z]*[0-9]*\])*','',copies_sold)
    return float(clean_value)*1_000_000

# print(convert_to_float("1 million[29]"))

def format_date(date):
    current_format = "%B %d, %Y"
    desired_format = "%m/%d/%Y"
    date_string = datetime.datetime.strptime(date,current_format)

    return date_string.strftime(desired_format)

# format_date("March 31, 2023")

# getting reviews from metacentric website
def create_reviews_url(link):
    clean_game_title = re.sub(r'[:,;.?\[\]]','',link).replace(' ','-').replace('--', '-').lower()
    return clean_game_title

create_reviews_url("1-2- Switch")


'1-2--switch'

In [33]:
sales_formatted_df = table_dataframe.assign(
    Copies_sold = lambda df: df['Copies sold'].map(lambda values: convert_to_float(values)),
    As_of = lambda df: df ['As of'].map(lambda values: format_date(values)),
    Release_date = lambda df: df ['Release date[a]'].map((lambda values: format_date(values)))
).drop(columns=['As of','Copies sold','Release date[a]'], axis=1)

sales_formatted_df

,Rank,Title,Genre(s),Developer(s),Publisher(s),Copies_sold,As_of,Release_date
0,1,Mario Kart 8 Deluxe,Kart racing,Nintendo EPD,Nintendo,53790000.0,03/31/2023,04/28/2017
1,2,Animal Crossing: New Horizons,Social simulation,Nintendo EPD,Nintendo,42210000.0,03/31/2023,03/20/2020
2,3,Super Smash Bros. Ultimate,Fighting,Bandai Namco StudiosSora Ltd.,Nintendo,31090000.0,03/31/2023,12/07/2018
3,4,The Legend of Zelda: Breath of the Wild,Action-adventure,Nintendo EPD,Nintendo,29810000.0,03/31/2023,03/03/2017
4,5,Pokémon Sword and Shield,Role-playing,Game Freak,The Pokémon CompanyNintendo,25820000.0,03/31/2023,11/15/2019
...,...,...,...,...,...,...,...,...
73,72,Fitness Boxing,Exergamerhythm,Imagineer,JP: ImagineerNA/PAL: Nintendo,1000000.0,09/08/2020,12/20/2018
74,72,Fitness Boxing 2: Rhythm and Exercise,Exergamerhythm,Imagineer,JP: ImagineerNA/PAL: Nintendo,1000000.0,12/09/2021,12/04/2020
75,72,Shin Megami Tensei V,Role-playing,Atlus,JP: AtlusNA: SegaPAL: Nintendo,1000000.0,04/18/2022,11/11/2021
76,72,Story of Seasons: Pioneers of Olive Town,Simulationrole-playing,Marvelous,Xseed Games,1000000.0,11/18/2021,02/25/2021
